## Get first 10000 rows modified

In [1]:
import json

formatted_strings = []

with open("train.jsonl", "r") as f:
    for j, line in enumerate(f):
        if j >= 10000:
            break
        # Parse the JSON data from the line
        data = json.loads(line.strip())
        rating = data['Rating']
        title = data['Title']
        review = data['Review']

        # If "Title: " appears in the review, trim off everything after its first occurrence.
        title_marker_index = review.find("Title: ")
        if title_marker_index != -1:
            review = review[:title_marker_index]

        # Format the string as required
        formatted_string = (
            f'"System prompt : Given the Rating and Title, you are required to generate the review" | '
            f'"Rating": {rating} | "Title": {title} | "Review": '
        )

        # Add the formatted string to the list
        formatted_strings.append(formatted_string)

print(f"Processed {len(formatted_strings)} lines.")


Processed 10000 lines.


In [2]:
formatted_strings[0]

'"System prompt : Given the Rating and Title, you are required to generate the review" | "Rating": 4 | "Title": No white background! It’s clear! | "Review": '

In [3]:
# from transformers import AutoModelForCausalLM, AutoTokenizer
# import torch

# save_directory = "."

# # Load the model with half precision if supported
# model = AutoModelForCausalLM.from_pretrained(save_directory, torch_dtype=torch.float16)
# tokenizer = AutoTokenizer.from_pretrained(save_directory)

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# print(f"Using device: {device}")

# model.to(device)
# model.eval()


# sample_prompt = ("System prompt: Given the Rating and Title, you are required to generate the review, "
#                  "Rating: 5, Title: Would definitely buy again, Review:")

# inputs = tokenizer(sample_prompt, return_tensors="pt", padding=True, truncation=True, max_length=128)
# inputs = {key: value.to(device) for key, value in inputs.items()}

# with torch.no_grad():
#     generated_ids = model.generate(**inputs, max_length=128, do_sample=True, top_k=50)
# generated_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
# print("Generated text:", generated_text)


## Load model with DP and perform 10000 inferences


In [4]:
import json
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# Directory where your model is saved
save_directory = "."

# Load the model with half precision if supported
model = AutoModelForCausalLM.from_pretrained(save_directory, torch_dtype=torch.float16)
tokenizer = AutoTokenizer.from_pretrained(save_directory)

# Set up the device: use CUDA if available, otherwise fallback to CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

model.to(device)
model.eval()


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Using device: cuda


LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): lora.Linear(
            (base_layer): Linear(in_features=4096, out_features=4096, bias=False)
            (lora_dropout): ModuleDict(
              (default): Dropout(p=0.05, inplace=False)
            )
            (lora_A): ModuleDict(
              (default): Linear(in_features=4096, out_features=8, bias=False)
            )
            (lora_B): ModuleDict(
              (default): Linear(in_features=8, out_features=4096, bias=False)
            )
            (lora_embedding_A): ParameterDict()
            (lora_embedding_B): ParameterDict()
            (lora_magnitude_vector): ModuleDict()
          )
          (k_proj): lora.Linear(
            (base_layer): Linear(in_features=4096, out_features=1024, bias=False)
            (lora_dropout): ModuleDict(
              (defau

In [5]:
# import time
# import json  # Import json to handle JSON serialization

# # Read formatted prompts from file. Each line should contain one formatted prompt.
# formatted_prompts = formatted_strings

# # Output file to save generated sequences in JSONL format
# output_file = "generated_sequences.jsonl"

# # Set batch size to 10 and initialize timing and batch results
# batch_size = 10
# results_batch = []
# batch_start_time = time.time()

# for i, prompt in enumerate(formatted_prompts):
#     # Tokenize the prompt
#     inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True, max_length=128)
#     inputs = {key: value.to(device) for key, value in inputs.items()}

#     # Generate text from the prompt
#     with torch.no_grad():
#         generated_ids = model.generate(**inputs, max_length=128, do_sample=True, top_k=50)
#     generated_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)

#     results_batch.append(generated_text)

#     # Save batch and compute time after every 10 iterations
#     if (i + 1) % batch_size == 0:
#         batch_end_time = time.time()
#         batch_time = batch_end_time - batch_start_time

#         # Append ge


In [7]:
import time

output_file = "generated_sequences.jsonl"
batch_size = 100
formatted_prompts = formatted_strings
# Process prompts in batches
num_prompts = len(formatted_prompts)
for batch_start in range(0, num_prompts, batch_size):
    batch_prompts = formatted_prompts[batch_start : batch_start + batch_size]

    batch_start_time = time.time()

    # Tokenize the entire batch
    inputs = tokenizer(batch_prompts, return_tensors="pt", padding=True, truncation=True, max_length=128)
    inputs = {key: value.to(device) for key, value in inputs.items()}

    # Generate text for the batch
    with torch.no_grad():
        generated_ids = model.generate(**inputs, max_length=128, do_sample=True, top_k=50)

    # Decode the generated sequences for each prompt
    batch_generated_texts = [tokenizer.decode(ids, skip_special_tokens=True) for ids in generated_ids]

    batch_end_time = time.time()
    batch_time = batch_end_time - batch_start_time

    # Write the generated outputs in JSONL format
    with open(output_file, "a") as outfile:
        for text in batch_generated_texts:
            json_line = json.dumps({"generated_text": text})
            outfile.write(json_line + "\n")

    print(f"Processed batch {(batch_start // batch_size) + 1} (prompts {batch_start} to {batch_start+len(batch_prompts)-1}). Time taken: {batch_time:.2f} seconds.")


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


Processed batch 1 (prompts 0 to 99). Time taken: 13.89 seconds.


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


Processed batch 2 (prompts 100 to 199). Time taken: 14.01 seconds.


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


...

Processed batch 97 (prompts 9600 to 9699). Time taken: 14.25 seconds.


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


Processed batch 98 (prompts 9700 to 9799). Time taken: 14.46 seconds.


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


Processed batch 99 (prompts 9800 to 9899). Time taken: 14.15 seconds.
Processed batch 100 (prompts 9900 to 9999). Time taken: 14.37 seconds.


## Load model without dp and run 10000 inferences

In [3]:
import json
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# Directory where your model is saved
save_directory = "./finetuned_no_dp"

# Load the model with half precision if supported
model = AutoModelForCausalLM.from_pretrained(save_directory, torch_dtype=torch.float16)
tokenizer = AutoTokenizer.from_pretrained(save_directory)

# Set up the device: use CUDA if available, otherwise fallback to CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

model.to(device)
model.eval()


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Using device: cuda


LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): lora.Linear(
            (base_layer): Linear(in_features=4096, out_features=4096, bias=False)
            (lora_dropout): ModuleDict(
              (default): Dropout(p=0.05, inplace=False)
            )
            (lora_A): ModuleDict(
              (default): Linear(in_features=4096, out_features=8, bias=False)
            )
            (lora_B): ModuleDict(
              (default): Linear(in_features=8, out_features=4096, bias=False)
            )
            (lora_embedding_A): ParameterDict()
            (lora_embedding_B): ParameterDict()
            (lora_magnitude_vector): ModuleDict()
          )
          (k_proj): lora.Linear(
            (base_layer): Linear(in_features=4096, out_features=1024, bias=False)
            (lora_dropout): ModuleDict(
              (defau

In [4]:
import time

output_file = "generated_sequences_no_dp.jsonl"
batch_size = 200
formatted_prompts = formatted_strings
# Process prompts in batches
num_prompts = len(formatted_prompts)
for batch_start in range(0, num_prompts, batch_size):
    batch_prompts = formatted_prompts[batch_start : batch_start + batch_size]

    batch_start_time = time.time()

    # Tokenize the entire batch
    inputs = tokenizer(batch_prompts, return_tensors="pt", padding=True, truncation=True, max_length=128)
    inputs = {key: value.to(device) for key, value in inputs.items()}

    # Generate text for the batch
    with torch.no_grad():
        generated_ids = model.generate(**inputs, max_length=128, do_sample=True, top_k=50)

    # Decode the generated sequences for each prompt
    batch_generated_texts = [tokenizer.decode(ids, skip_special_tokens=True) for ids in generated_ids]

    batch_end_time = time.time()
    batch_time = batch_end_time - batch_start_time

    # Write the generated outputs in JSONL format
    with open(output_file, "a") as outfile:
        for text in batch_generated_texts:
            json_line = json.dumps({"generated_text": text})
            outfile.write(json_line + "\n")

    print(f"Processed batch {(batch_start // batch_size) + 1} (prompts {batch_start} to {batch_start+len(batch_prompts)-1}). Time taken: {batch_time:.2f} seconds.")


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


Processed batch 1 (prompts 0 to 199). Time taken: 24.01 seconds.


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


Processed batch 2 (prompts 200 to 399). Time taken: 23.07 seconds.


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


...

Processed batch 47 (prompts 9200 to 9399). Time taken: 22.81 seconds.


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


Processed batch 48 (prompts 9400 to 9599). Time taken: 24.72 seconds.


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


Processed batch 49 (prompts 9600 to 9799). Time taken: 24.32 seconds.
Processed batch 50 (prompts 9800 to 9999). Time taken: 24.16 seconds.
